In [3]:
import boto3
import pprint
from botocore.client import Config
from IPython.core.display import HTML
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [4]:
# restart kernel
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

pp = pprint.PrettyPrinter(indent=2)
region = "us-east-1"
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})

bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              region_name=region,
                              config=bedrock_config,
                                    )
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

In [5]:
bucket_name = "bedrock-demo-01-09"
prefix_file_name = "TheEvolutionofRAG.pdf"
document_s3_uri = f's3://{bucket_name}/{prefix_file_name}'

In [6]:
def retrieveAndGenerate(input, model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
    model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'
    return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    "sources": [
                        {
                            "sourceType": "S3",
                            "s3Location": {
                                "uri": document_s3_uri
                            }
                        }
                    ]
                }
            }
        )

In [7]:
query = "Summarize the document"
response = retrieveAndGenerate(input=query)
generated_text = response['output']['text']
pp.pprint(generated_text)

('The document traces the evolution of Retrieval-Augmented Generation (RAG) in '
 'AI from its foundational inception to its current state as a complex, '
 'modular, and agentic system. It outlines four key stages:\n'
 '\n'
 '1) Naive RAG: The basic "retrieve-then-generate" pipeline where relevant '
 'information is retrieved from an external source and provided to a language '
 'model to generate a response.\n'
 '\n'
 '2) Advanced RAG: Optimizations and refinements to the pipeline, such as '
 'improved retrieval techniques, re-ranking models, and context compression, '
 'to enhance accuracy and reliability.\n'
 '\n'
 '3) Modular RAG: A shift towards a flexible, modular architecture where '
 'specialized components like retrievers, re-rankers, and generators are '
 'orchestrated by an intelligent agent to handle complex queries and perform '
 'multi-step reasoning.\n'
 '\n'
 '4) Agentic RAG: The current frontier where autonomous agents plan and '
 'execute multi-step retrieval strategi